## AML 01
***

### Data Preparation

In [1]:
from sklearn.datasets import load_digits
from data_preparation import data_preparation

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
digits = load_digits()
print(digits.keys())

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])


In [3]:
data = digits['data']
images = digits['images']
target = ['target']
target_names = ['target_names']

In [4]:
X, y = data_preparation(digits, filter=[3, 8])

In [5]:
lambdas = [10 ** i for i in range(-5, 5)]

In [6]:
accuracies = []
print("Lambda\t\t Accuracy")
print("=========================")
for lmbda in lambdas:
    LR = LogisticRegression(C=lmbda, solver='liblinear')
    accuracy = cross_val_score(LR, X, y, cv=5)
    accuracies.append(np.mean(accuracy))
    print("{} \t\t {:.5f}".format(lmbda, accuracies[-1]))

Lambda		 Accuracy
1e-05 		 0.94937
0.0001 		 0.95227
0.001 		 0.94953
0.01 		 0.95802
0.1 		 0.96365
1 		 0.96643
10 		 0.96362
100 		 0.96084
1000 		 0.95802
10000 		 0.95802
